In [2]:
import json

with open("/tmp/waxholm_raw_lexicon.json") as lexjson:
    data = json.load(lexjson)

In [4]:
def simplify_stops(text):
    text = text.replace("Kk", "K")
    text = text.replace("Gg", "G")
    text = text.replace("Dd", "D")
    text = text.replace("Tt", "T")
    text = text.replace("Bb", "B")
    text = text.replace("Pp", "P")
    text = text.replace("k", "K")
    text = text.replace("g", "G")
    text = text.replace("d", "D")
    text = text.replace("t", "T")
    text = text.replace("b", "B")
    text = text.replace("p", "P")
    return text


In [7]:
simple_matched_words = {}
simple_matched_sentences = []
rest = []
for item in data:
    if item["phoneme"] == simplify_stops(item["labels"]):
        text_parts = item["text"].split(" ")
        phone_parts = item["phoneme"].split(" ")
        if len(text_parts) == 1 and len(phone_parts) == 1:
            if item["text"] not in simple_matched_words:
                simple_matched_words[item["text"]] = set()
            simple_matched_words[item["text"]].add(item["phoneme"])
        else:
            simple_matched_sentences.append(item)
    else:
        rest.append(item)

In [10]:
simple_matched_words

{'slut': {"SL'U:T"},
 'tack': {"T'AK"},
 'idag': {"ID'A:G"},
 'ikväll': {"IKV'ÄL"},
 'härifrån': {"H'Ä3RIFRÅN"},
 'hej': {"H'EJ"},
 'hotell': {"HOT'EL"},
 'tisdag': {"T'I:SDA"},
 'fredag': {"FR'E:DA"},
 'onsdag': {"'ONSDA"},
 'ingenting': {"INGENT'ING"},
 'söndag': {"S'ÖNDA"},
 'lördag': {"L'Ö32DA"},
 'måndag': {"M'ÅNDA"},
 '5': {"F'EM"}}

In [14]:
not_as_simple = []
for item in simple_matched_sentences:
    text_parts = item["text"].split(" ")
    phone_parts = item["phoneme"].split(" ")
    if len(text_parts) == len(phone_parts):
        for i in range(0, len(text_parts)):
            if text_parts[i] not in simple_matched_words:
                simple_matched_words[text_parts[i]] = set()
            simple_matched_words[text_parts[i]].add(phone_parts[i])
    else:
        not_as_simple.append(item)

In [18]:
simple_matched_words

{'slut': {"SL'U:T"},
 'tack': {"T'AK"},
 'idag': {"ID'A:G"},
 'ikväll': {"IKV'ÄL"},
 'härifrån': {"H'Ä3RIFRÅN"},
 'hej': {"H'EJ"},
 'hotell': {"HOT'EL"},
 'tisdag': {"T'I:SDA"},
 'fredag': {"FR'E:DA"},
 'onsdag': {"'ONSDA"},
 'ingenting': {"INGENT'ING"},
 'söndag': {"S'ÖNDA"},
 'lördag': {"L'Ö32DA"},
 'måndag': {"M'ÅNDA"},
 '5': {"F'EM"},
 'kväll': {"KV'ÄL"},
 'stan': {"ST'A:N"}}

In [20]:
not_as_simple

[]

In [25]:
def simplify_phoneme(text):
    text = text.replace("+", "")
    text = text.replace("hy", "#")
    return text

In [28]:
other_words = []
for item in rest:
    def update_words(word, phone):
        if word not in simple_matched_words:
            simple_matched_words[word] = set()
        simple_matched_words[word].add(phone)

    text_parts = item["text"].split(" ")
    phone_parts = item["phoneme"].split(" ")
    label_parts = item["labels"].split(" ")
    if len(text_parts) == len(phone_parts) == len(label_parts):
        for i in range(0, len(text_parts)):
            simple_label = simplify_stops(label_parts[i])
            simple_phoneme = simplify_phoneme(phone_parts[i])
            if simple_label == simple_phoneme:
                update_words(text_parts[i], simple_phoneme)
            else:
                other_words.append((text_parts[i], simple_phoneme, simple_label))
    

In [35]:
other_words

[('Lediga', 'L"E:D\'IGA', 'L"E:DIGA'),
 ('och', "'Å:", "'ÅK"),
 ('utvilade', '"U:T#V\'I:LADE', '"U:T#V`I:LADE0'),
 ('tittade', 'T"IT\'ADE', 'T"ITADE0'),
 ('de', "D'OM", "D'ÅM"),
 ('föreställningen', 'F"Ö3RE#ST\'ÄLNINGEN', 'F"Ö3RE#ST`ÄLNINGE0N'),
 ('timme', 'T"IM\'E', 'T"IME0'),
 ('stockholm', 'ST"ÅK#\'ÅLM', 'ST"ÅK#`ÅLM'),
 ('Lila', 'L"I:L\'A', "L'I:LA"),
 ('stolar', 'ST"O:L\'AR', 'ST"O:LAR'),
 ('salen', "S'A:LEN", "S'A:LE0N"),
 ('jag', "J'A:G", "J'A:"),
 ('åka', '"Å:K\'A', '"Å:KA'),
 ('jag', "J'A:G", "J'A:"),
 ('letar', 'L"E:T\'AR', 'L"E:TAR'),
 ('efter', "'EFTÄ4R", "'EFTE0R"),
 ('sandhamn', 'S"AND#H\'AMN', 'S"ANDv#H`AMN'),
 ('Lediga', 'L"E:D\'IGA', 'L"E:DIGA'),
 ('och', "'Å:", "'ÅK"),
 ('utvilade', '"U:T#V\'I:LADE', '"U:T#V`I:LADE0'),
 ('tittade', 'T"IT\'ADE', 'T"ITADE0'),
 ('de', "D'OM", "D'ÅM"),
 ('föreställningen', 'F"Ö3RE#ST\'ÄLNINGEN', 'F"Ö3RE#ST`ÄLNINGE0N'),
 ('timme', 'T"IM\'E', 'T"IME0'),
 ('strömkajen', 'STR"ÖM#K\'AJEN', 'STR"ÖM#K`AJE0N'),
 ('Sprakande', 'SPR"A:K\'ANDE', 'SPR

In [41]:
chars = set()
for word in other_words:
    for char in word[1]:
        chars.add(char)

In [42]:
chars

{'"',
 '#',
 "'",
 ',',
 '0',
 '2',
 '3',
 '4',
 ':',
 'A',
 'B',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 'Y',
 'a',
 'b',
 'd',
 'g',
 'h',
 'k',
 'o',
 'p',
 'r',
 'v',
 '~',
 'Ä',
 'Å',
 'Ö',
 'ö'}

In [90]:
def segment_label(label, skip_pause=True):
    phones = []
    i = 0
    while i < len(label):
        start_i = i
        end_i = i
        if label[i:i+2] in ["NG", "E0", "SJ", "TJ", "kl", "sm", "kl", "pa"]:
            phones.append(label[i:i+2])
            i += 2
        elif label[i:i+2] == "p:":
            if not skip_pause:
                phones.append("p:")
            i += 2
        elif label[i:i+1] == "#":
            i += 1
        else:
            if label[i:i+1] in ["'", "`", "\"", "2", "~"]:
                i += 1
                end_i += 1
            if label[i+1:i+2] in [":", "3", "4"]:
                end_i += 1
            phones.append(label[start_i:end_i+1])
            i = end_i + 1
    return phones

In [95]:
for sword in simple_matched_words:
    for item in simple_matched_words[sword]:
        print(f"{sword}\t{' '.join(segment_label(item))}")

slut	S L 'U: T
tack	T 'A K
idag	I D 'A: G
ikväll	I K V 'Ä L
härifrån	H 'Ä3 R I F R Å N
hej	H 'E J
hotell	H O T 'E L
tisdag	T 'I: S D A
fredag	F R 'E: D A
onsdag	'O N S D A
ingenting	I NG E N T 'I NG
söndag	S 'Ö N D A
lördag	L 'Ö3 2D A
måndag	M 'Å N D A
5	F 'E M
kväll	K V 'Ä L
stan	S T 'A: N
på	P 'Å:
i	'I:
en	'E N
från	F R 'Å: N
bärs	B 'Ä3 2S
in	'I N
vill	V 'I L
ett	'E T
oss	'Å S
var	V 'A: R
finns	F 'I N S
har	H 'A: R
till	T 'I L
då	D 'Å:
nöjd	N 'Ö J D
båt	B 'Å: T
går	G 'Å: R
jag	J 'A: G
jag	J 'A:
ut	'U: T
som	S 'Å M
ihjäl	I J 'Ä: L
och	'Å:
bli	B L 'I:
bra	B R 'A:
Där	D 'Ä3 R
hus	H 'U: S
att	'A T
vi	V 'I:
om	'Å M
skärgården	SJ 'Ä3 R G Å: 2N
tror	T R 'O: R
förstår	F Ö4 2S 2T 'Å: R
bo	B 'O:
nu	N 'U:
ja	J 'A:
ramsöberg	R A M S Ö: B 'Ä4 R J
istället	I S T "Ä L E0 T
dit	D 'I: T
för	F 'Ö3 R
ta	T 'A:
mig	M 'E J
XtvekX	ö h
så	S 'Å:
med	M 'E:
där	D 'Ä3 R
tillbaka	T I B 'A: K A
tillbaka	T I B 'A: K S
varifrån	V A R I F R 'Å: N
varifrån	V A: 2T I F R 'Å: N
varifrån	V A: R I F R 'Å: N
camping	K 'A 

In [92]:
simple_matched_words

{'slut': {"SL'U:T"},
 'tack': {"T'AK"},
 'idag': {"ID'A:G"},
 'ikväll': {"IKV'ÄL"},
 'härifrån': {"H'Ä3RIFRÅN"},
 'hej': {"H'EJ"},
 'hotell': {"HOT'EL"},
 'tisdag': {"T'I:SDA"},
 'fredag': {"FR'E:DA"},
 'onsdag': {"'ONSDA"},
 'ingenting': {"INGENT'ING"},
 'söndag': {"S'ÖNDA"},
 'lördag': {"L'Ö32DA"},
 'måndag': {"M'ÅNDA"},
 '5': {"F'EM"},
 'kväll': {"KV'ÄL"},
 'stan': {"ST'A:N"},
 'på': {"P'Å:"},
 'i': {"'I:"},
 'en': {"'EN"},
 'från': {"FR'Å:N"},
 'bärs': {"B'Ä32S"},
 'in': {"'IN"},
 'vill': {"V'IL"},
 'ett': {"'ET"},
 'oss': {"'ÅS"},
 'var': {"V'A:R"},
 'finns': {"F'INS"},
 'har': {"H'A:R"},
 'till': {"T'IL"},
 'då': {"D'Å:"},
 'nöjd': {"N'ÖJD"},
 'båt': {"B'Å:T"},
 'går': {"G'Å:R"},
 'jag': {"J'A:", "J'A:G"},
 'ut': {"'U:T"},
 'som': {"S'ÅM"},
 'ihjäl': {"IJ'Ä:L"},
 'och': {"'Å:"},
 'bli': {"BL'I:"},
 'bra': {"BR'A:"},
 'Där': {"D'Ä3R"},
 'hus': {"H'U:S"},
 'att': {"'AT"},
 'vi': {"V'I:"},
 'om': {"'ÅM"},
 'skärgården': {"SJ'Ä3RGÅ:2N"},
 'tror': {"TR'O:R"},
 'förstår': {"FÖ42S2T'Å:R